# Algoritmo de Otimização Bayesiana

## Importanto as bibliotecas

In [3]:
import numpy as np
from math import sin
from math import pi
from sklearn.gaussian_process import GaussianProcessRegressor
from warnings import catch_warnings
from warnings import simplefilter
from scipy.stats import norm

## Objective Function f(x_i)

In [4]:
# y = x^2 * sin(5 * PI * x)^6
# ruído gaussino com a média= 0 e std= 0.1 
# x: número real que pertence ao intervalo [0,1]
def objective(x, noise=0.1):
    noise = np.random.normal(loc=0, scale=noise)
    return (x**2 * sin(5 * pi * x)**6.0) + noise

## Surrogate Function P(f|D)

In [ ]:
def surrogate(model, X):
    with catch_warnings():
        simplefilter("ignore")
        return model.predict(X, return_std=True)

## Acquisition Fuction

In [ ]:
def acquisition(X, Xsamples, model):
    # Calcular a melhor pontuação (score) alcançada por uma entrada
    y_pred, std_pred = surrogate(model, X)
    best = max(y_pred)
    
    # Calcular a média e o desvio dos scores através da surrogate 
    mu, std = surrogate(model, Xsamples)
    mu = mu[:, 0]
    
    # Calcula a função de distribuição cumulativa das probabilidade de melhoria 
    probs = norm.cdf((mu - best) / (std+1E-9))
    
    return probs

In [ ]:
def opt_acquisition(X, model):
    # Gera amostras de entrada aleatoriamente
    Xsamples = np.random.random(100)
    Xsamples = Xsamples.reshape(len(Xsamples), 1)
    
    # Gera a distribuição de probabilidade de melhoria para X 
    scores = acquisition(X, Xsamples, model)
    
    # Captura o index da amostra que possui maior probabilidade de melhoria em relação ao atual 
    ix = np.argmax(scores)
    
    return Xsamples[ix, 0]

## Passo a Passo da otimização Bayesiana

In [ ]:
# 1. Gerar os dados de entrada (X) e os respectivos scores (y)
X = np.random.random(100)
y = np.asarray([objective(x) for x in X])

# reshape 1d(100,) -> 2d(100,1) 
X = X.reshape(len(X), 1)
y = y.reshape(len(y), 1)

# Utilizar o modelo de regressão Gaussiana para fitar os dados
model = GaussianProcessRegressor()
model.fit(X, y)

# Realizar o processo de otimização Bayesiana
for i in range(50):
    # 1. Selecionar a próxima amostra a ser analisada:
    x = opt_acquisition(X, model)
    
    # 2. Avaliar o score da amostra com a função objetivo:
    score_real = objective(x)
    
    score_est, std = surrogate(model, [[x]])
    print('>x=%.3f, f()=%3f, actual=%.3f' % (x, score_est, score_real))
    
    # Adiciona o novo par (amostra, score) 
    X = np.append(X, [[x]], axis = 0)
    y = np.append(y, [[score_real]], axis = 0)
                  
    # 3. Treino novamente o modelo com os dados atualizados e, consequentemente atualiza a estimativa da função surrogate 
    model.fit(X, y)

>x=0.806, f()=0.275057, actual=-0.097
>x=0.735, f()=0.185613, actual=0.083
>x=0.011, f()=-0.013524, actual=0.100
>x=0.659, f()=0.142586, actual=0.139
>x=0.571, f()=0.111097, actual=0.158
>x=0.212, f()=-0.015196, actual=-0.044
>x=0.888, f()=0.310249, actual=0.842
>x=0.889, f()=0.349354, actual=0.790
>x=0.409, f()=0.026145, actual=0.039
>x=0.848, f()=0.345139, actual=0.196
>x=0.265, f()=-0.023890, actual=0.168
>x=0.503, f()=0.083408, actual=0.096
>x=0.891, f()=0.371529, actual=0.837
>x=0.227, f()=-0.013573, actual=-0.030
>x=0.890, f()=0.400801, actual=0.666
>x=0.187, f()=-0.011045, actual=-0.039
>x=0.250, f()=-0.018314, actual=0.115
>x=0.519, f()=0.093791, actual=0.187
>x=0.891, f()=0.416383, actual=0.846
>x=0.119, f()=-0.010870, actual=-0.019
>x=0.892, f()=0.440359, actual=0.710
>x=0.703, f()=0.140367, actual=0.506
>x=0.849, f()=0.405226, actual=0.101
>x=0.893, f()=0.440132, actual=0.773
>x=0.893, f()=0.456511, actual=0.728
>x=0.910, f()=0.457198, actual=0.586
>x=0.098, f()=-0.015259, a

In [ ]:
ix = np.argmax(y)
print('Best Result: x=%.3f, y=%.3f' % (X[ix], y[ix]))

Best Result: x=0.893, y=0.882


# Ajuste de hiperparâmetros com otimização bayesiana (Scikit-Optimize)

## Importando as bibliotecas

In [7]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
from sklearn.model_selection import cross_val_score
import time

## Carragando os dados

In [8]:
X,y = datasets.load_breast_cancer(return_X_y=True)
print(X.shape, y.shape)

(569, 30) (569,)


In [9]:
model = RandomForestClassifier()

In [10]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [11]:
search_space = {
    'n_estimators': (90,130),
    'max_depth': (4,20),
}

In [12]:
opt = BayesSearchCV(model, search_space, n_iter=20, cv=5)

TypeError: __init__() got an unexpected keyword argument 'iid'

In [ ]:
start = time.time()
opt.fit(X, y)
end = time.time()

C:\Users\joseb\AppData\Local\Programs\Python\Python38-32\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\joseb\AppData\Local\Programs\Python\Python38-32\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\joseb\AppData\Local\Programs\Python\Python38-32\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\joseb\AppData\Local\Programs\Python\Python38-32\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\joseb\AppData\Local\Programs\Python\Python38-32\lib\site-packages\skopt\opt

In [ ]:
best_estimator = opt.best_estimator_
print(best_estimator)
cvs = cross_val_score(best_estimator, X, y, cv=5)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=116,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [ ]:
print("Score best parameters: ", opt.best_score_)
print("Best parameters: ", opt.best_params_)
print("5-fold cross-validation Score: ", cvs.mean())
print("Time elapsed: ", end - start)

Score best parameters:  0.968365553602812
Best parameters:  OrderedDict([('max_depth', 19), ('n_estimators', 116)])
5-fold cross-validation Score:  0.9631113181183046
Time elapsed:  58.61742067337036


Atividade(2.5): Realizar a otimização Bayesiana:
<ol>
        <li> Além do RandomForest, utilizando outro modelo de classificação (a sua escolha). Dica: definir no espaço de busca pelo menos 3 hiperparâmetros de cada modelo; </li> 
        <li> Extrapolar o exemplo acima para avaliar a acurácia de classificação para um conjunto de teste (divisão treino/teste a sua escolha);</li> 
        <li> Realizar a otimização Bayesiana utilizando a biblioteca HyperOpt;</li> 
        <li> Comparar os resultados das otimizações Bayesianas (Scikit-Optimize e HyperOpt) com aqueles obtidos pelo GridSearch e RandomSearch: número de iterações, número de pastas (cv), melhor score obtido para conj. de teste e tempo de busca. Dica: é interessante que o número de iterações seja o mesmo para todas as abordagens </li>    
</ol>